# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kuito,-12.3833,16.9333,49.50,41,11,4.00,AO,1627624643
1,1,naze,28.3667,129.4833,83.32,81,94,1.66,JP,1627624644
2,2,port-gentil,-0.7193,8.7815,76.96,83,75,10.36,GA,1627624644
3,3,rikitea,-23.1203,-134.9692,71.46,74,1,6.11,PF,1627624644
4,4,busselton,-33.6500,115.3333,52.09,63,84,23.53,AU,1627624645


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [28]:
# Create Heatmap for Humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
# Narrow down cities that has max temperature lower than 80 degrees but higher than 70,  
# Wind speed less than 10 mph and 
# Zero cloudiness.

city_temp_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
city_temp_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,barga,44.0742,10.4808,74.26,61,0,1.25,IT,1627624645
204,204,belen,36.4891,36.2233,76.03,47,0,7.14,TR,1627624704
233,233,oktyabrskoye,43.0645,44.7417,71.74,68,0,2.24,RU,1627624712
234,234,ilinskiy,55.6190,38.1182,73.31,78,0,0.00,RU,1627624712
336,336,obodivka,48.4051,29.2477,71.19,72,0,7.38,UA,1627624742
351,351,perpignan,42.7500,2.7500,70.25,84,0,4.61,FR,1627624683
439,439,asau,46.4333,26.4000,78.40,61,0,1.32,RO,1627624774
444,444,bom jesus da lapa,-13.2550,-43.4181,70.54,44,0,8.90,BR,1627624776
484,484,awbari,26.5921,12.7805,79.48,26,0,7.05,LY,1627624788
547,547,zhezkazgan,47.8043,67.7144,77.41,31,0,5.01,KZ,1627624812


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df = city_temp_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
5,barga,IT,44.0742,10.4808,
204,belen,TR,36.4891,36.2233,
233,oktyabrskoye,RU,43.0645,44.7417,
234,ilinskiy,RU,55.6190,38.1182,
336,obodivka,UA,48.4051,29.2477,
351,perpignan,FR,42.7500,2.7500,
439,asau,RO,46.4333,26.4000,
444,bom jesus da lapa,BR,-13.2550,-43.4181,
484,awbari,LY,26.5921,12.7805,
547,zhezkazgan,KZ,47.8043,67.7144,


In [23]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Get hotel nnames and skip anny missing data
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
5,barga,IT,44.0742,10.4808,
204,belen,TR,36.4891,36.2233,
233,oktyabrskoye,RU,43.0645,44.7417,
234,ilinskiy,RU,55.6190,38.1182,
336,obodivka,UA,48.4051,29.2477,
351,perpignan,FR,42.7500,2.7500,
439,asau,RO,46.4333,26.4000,
444,bom jesus da lapa,BR,-13.2550,-43.4181,
484,awbari,LY,26.5921,12.7805,
547,zhezkazgan,KZ,47.8043,67.7144,


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))